# Introduction to Neural Networks: Classification

Author: Pierre Nugues

## A Dataset: *Salammbô* in French and English

### Understanding the Dataset

The counts of letters and _A_ broken down by chapter in *Salammbô* in French and its translation in English. The $\mathbf{X}$ matrix contains the counts and $y$ the language; French: 1 and English: 0.

In [1]:
import numpy as np
np.random.seed(0)

X = np.array(
    [[35680, 2217], [42514, 2761], [15162, 990], [35298, 2274],
     [29800, 1865], [40255, 2606], [74532, 4805], [37464, 2396],
     [31030, 1993], [24843, 1627], [36172, 2375], [39552, 2560],
     [72545, 4597], [75352, 4871], [18031, 1119], [36961, 2503],
     [43621, 2992], [15694, 1042], [36231, 2487], [29945, 2014],
     [40588, 2805], [75255, 5062], [37709, 2643], [30899, 2126],
     [25486, 1784], [37497, 2641], [40398, 2766], [74105, 5047],
     [76725, 5312], [18317, 1215]
     ], dtype='float64')

y = np.array(
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
     1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

### Visualizing the Data

In [2]:
import matplotlib.pyplot as plt

X_english, X_french = np.split(X, 2)

en = plt.scatter(X_english[:,0], X_english[:,1], c='r', marker='x')
fr = plt.scatter(X_french[:,0], X_french[:,1], c='b', marker='x')
plt.title("Salammbô")
plt.xlabel("Letter count")
plt.ylabel("A count")
plt.legend((en, fr), ('English', 'French'), loc='lower right', scatterpoints=1)
plt.show()

<Figure size 640x480 with 1 Axes>

It is probably possible to determine the language using the counts

## Building a Larger Dataset

Let us extend the dataset to cover all the paragraphs, where we will count all the letters.<br/> Structure of the dataset:<br/>

|Obs.   | *x*                              |class   |
|-------|----------------------------------|--------|
|para_1 | a_counts, b_counts, c_counts, ...| English|
|para_2 |a_counts, b_counts, c_counts, ... |English |
|...    |...                               |...    |
|para_1000| a_counts, b_counts, c_counts, ...|French|
|para_1001| a_counts, b_counts, c_counts, ...| French|
|...      |...                               |...     |

### A Function to Count Characters

In [3]:
def count_chars(string):
    counts = {}
    for char in string:
        if char.isalpha():
            if char in counts:
                counts[char] += 1
            else:
                counts[char] = 1
    return counts

### We Extract the Counts

In [4]:
salammbo_en = open('../../EDAN20/programs/corpus/Salammbo/salammbo_en.txt').read().strip().lower()
salammbo_fr = open('../../EDAN20/programs/corpus/Salammbo/salammbo.txt').read().strip().lower()

para_en = filter(None, salammbo_en.split('\n'))
para_fr = filter(None, salammbo_fr.split('\n'))

counts_en = list(filter(None, map(count_chars, para_en)))
counts_fr = list(filter(None, map(count_chars, para_fr)))
counts = counts_en + counts_fr
counts[:5]

[{'s': 1, 'a': 2, 'l': 1, 'm': 2, 'b': 1, 'ô': 1},
 {'b': 2,
  'y': 1,
  'g': 1,
  'u': 2,
  's': 1,
  't': 2,
  'a': 2,
  'v': 1,
  'e': 2,
  'f': 1,
  'l': 1,
  'r': 1},
 {'c': 1, 'h': 1, 'a': 1, 'p': 1, 't': 1, 'e': 1, 'r': 1, 'i': 1},
 {'c': 1, 'h': 1, 'a': 1, 'p': 1, 't': 1, 'e': 1, 'r': 1, 'i': 2},
 {'c': 1, 'h': 1, 'a': 1, 'p': 1, 't': 1, 'e': 1, 'r': 1, 'i': 3}]

### Formatting the Dataset

We associate the characters with indices

In [5]:
from sklearn.feature_extraction import DictVectorizer

dict_vec = DictVectorizer(sparse=False)
dict_vec.fit(counts)
dict_vec.vocabulary_

{'a': 0,
 'b': 1,
 'c': 2,
 'd': 3,
 'e': 4,
 'f': 5,
 'g': 6,
 'h': 7,
 'i': 8,
 'j': 9,
 'k': 10,
 'l': 11,
 'm': 12,
 'n': 13,
 'o': 14,
 'p': 15,
 'q': 16,
 'r': 17,
 's': 18,
 't': 19,
 'u': 20,
 'v': 21,
 'w': 22,
 'x': 23,
 'y': 24,
 'z': 25,
 'à': 26,
 'â': 27,
 'æ': 28,
 'ç': 29,
 'è': 30,
 'é': 31,
 'ê': 32,
 'ë': 33,
 'î': 34,
 'ï': 35,
 'ô': 36,
 'ù': 37,
 'û': 38,
 'œ': 39}

We convert the dictionaries into matrices

In [6]:
X_en = dict_vec.transform(counts_en)
X_fr = dict_vec.transform(counts_fr)
print(X_en.shape)
print(X_fr.shape)

(1971, 40)
(2015, 40)


We stack the French and English datasets

In [7]:
X = np.vstack((X_en, X_fr))
y = [0] * len(counts_en) + [1] * len(counts_fr)
X[:5,:]

array([[2., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 2., 0., 0., 0.,
        0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 1., 0., 0., 0.],
       [2., 2., 0., 0., 2., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 1., 1., 2., 2., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 1., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 1.,
        0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 1., 0., 1., 0., 0., 1., 2., 0., 0., 0., 0., 0., 0., 1.,
        0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 1., 0., 1., 0., 0., 1., 3., 0., 0., 0., 0., 0., 0., 1.,
        0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.]])

We reduce the number of parameters

In [8]:
X = X[:, :3]

### Training and Validation Sets

#### We shuffle the indices

In [9]:
indices = list(range(X.shape[0]))
np.random.shuffle(indices)
print(indices[:10])
X = X[indices, :]
y = np.array(y)[indices]

[2952, 1523, 3945, 3605, 545, 1465, 3856, 1390, 587, 1276]


#### We split the dataset

In [10]:
training_examples = int(X.shape[0] * 0.8)

X_train = X[:training_examples, :]
y_train = y[:training_examples]

X_val = X[training_examples:, :]
y_val = y[training_examples:]

### Standardizing the Dataset

We standardize the dataset. This is very significant for the final result

In [11]:
from sklearn.preprocessing import StandardScaler

std = StandardScaler()
std.fit(X_train)

X_train = std.transform(X_train)
X_val = std.transform(X_val)
X_train[:3, :]

array([[ 0.52851591, -0.1155739 ,  0.06498621],
       [ 0.52851591,  0.73273673,  0.37512997],
       [ 0.52851591, -0.39834411,  0.68527373]])

## A Simple Feed-Forward Network

### The Model

We create a simple architecture. This is simply logistic regression

In [12]:
from keras import models
from keras import layers

model = models.Sequential()

model.add(layers.Dense(1, input_dim=X.shape[1], activation='sigmoid'))

# Fitting the network
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])
model.summary()

Using TensorFlow backend.


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1)                 4         
Total params: 4
Trainable params: 4
Non-trainable params: 0
_________________________________________________________________


We record the updates

In [13]:
import keras
from keras.callbacks import TensorBoard
callbacks = [
    keras.callbacks.TensorBoard(
        log_dir='tb_log_folder',
        histogram_freq=10,
) ]

And we fit the model

In [14]:
history = model.fit(X_train, y_train, 
                    epochs=100, 
                    batch_size=32, 
                    validation_data=(X_val, y_val), 
                    verbose=0,
                    callbacks=callbacks)

### The Model Weights

In [15]:
model.get_weights()

[array([[ 0.48772746],
        [-1.3212544 ],
        [ 0.7881461 ]], dtype=float32), array([0.03683845], dtype=float32)]

### Visualizing The Training Process

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()


### Predicting and Evaluating

In [ ]:
y_predicted = model.predict(X_val)
print(y_predicted[:10])
print(y_val[:10])

# evaluate the model
scores = model.evaluate(X_val, y_val)
print('Scores:', scores)
"\n%s: %.2f%%" % (model.metrics_names[1], scores[1] * 100)

### Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix
y_pred = []
for i in y_predicted:
    if i < 0.5:
        y_pred.append([0])
    else:
        y_pred.append([1])
confusion_matrix(y_val, y_pred)

## A More Complex Model

We build a more complex model with two layers

In [ ]:
model = models.Sequential()

model.add(layers.Dense(20, input_dim=X.shape[1], activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))

# Fitting the network
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])
model.summary()

And we fit it

In [ ]:
history = model.fit(X_train, y_train, epochs=20, batch_size=32, 
                    validation_data=(X_val, y_val), verbose=0)

### Visualizing The Training Process

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

### Predicting

In [ ]:
y_predicted = model.predict(X_val)
print(y_predicted[:10])

# evaluate the model
scores = model.evaluate(X_val, y_val)
print(scores)
"\n%s: %.2f%%" % (model.metrics_names[1], scores[1] * 100)

### Confusion Matrix

In [ ]:
y_pred = []
for i in y_predicted:
    if i < 0.5:
        y_pred.append([0])
    else:
        y_pred.append([1])
confusion_matrix(y_val, y_pred)

### For a real system, see the _Compact Language Detector v3_: https://github.com/google/cld3